In [1]:
#Process log file into a data frame format

import pandas as pd
import re
import datetime
import os
import numpy as np

#os.chdir(
#    "/Users/sophiabulcock/Desktop/MDS-BLOCK-6/Data-586"
 #   "C:/Users/Graha/OneDrive/UBC_MDS_Lectures/data_586/project"
#)

date_ = []
time_ = []
pid_ = []
level_ = []
component_ = []
content_ = []
contentType_ = []
parsedData = []

#with open("HDFS_2k.log", "r") as file:
with open("HDFS_1.tar", "r") as file:
    for line in file:

        #Get all odered data
        columns1_4 = line.split(":", 1)[0].strip()

        #Remove all rows with an inproper format
        try:
            columns1_4.split(' ')[0]
            columns1_4.split(' ')[1]
            columns1_4.split(' ')[2]
            columns1_4.split(' ')[3]
            columns1_4.split(' ')[4]
            date = columns1_4.split(' ')[0]
            time = columns1_4.split(' ')[1]
            pid = columns1_4.split(' ')[2]
            level = columns1_4.split(' ')[3]
            component = columns1_4.split(' ')[4]
        except:
            pass


        # Get colon with mixed data 
        content = line.split(":", 1)[1].strip()  # after colon
        list1 = (line.split(":", 1)[1].strip()).split(" ", 3)[0:2]
        contentType=' '.join(list1)
        
        date_.append(date)
        time_.append(time)
        pid_.append(pid)
        level_.append(level)
        component_.append(component)
        content_.append(content)
        contentType_.append(contentType)

# Collect and filter data
parsedData = pd.DataFrame(
    {
        "Date": date_,
        "Time": time_,
        "PID": pid_,
        "Level": level_,
        "Component": component_,
        "Content": content_,
        "TypeOfContent": contentType_,
    }
)
parsedData["datetime_id"] = parsedData["Date"] + parsedData["Time"]
parsedData["Tag"] = parsedData["TypeOfContent"]

parsedData1 = parsedData[
    (parsedData["Tag"] == "PacketResponder 0")
    | (parsedData["Tag"] == "PacketResponder 1")
    | (parsedData["Tag"] == "PacketResponder 2")
    | (parsedData["Tag"] == "BLOCK* NameSystem.addStoredBlock:")
    | (parsedData["Tag"] == "BLOCK* NameSystem.delete:")
    | (parsedData["Tag"] == "BLOCK* NameSystem.allocateBlock:")
    | (parsedData["Tag"] == "BLOCK* ask")
    | (parsedData["Tag"] == "Received block")
    | (parsedData["Tag"] == "Receiving block")
    | (parsedData["Tag"] == "Verification succeeded")
    | (parsedData["Tag"] == "Deleting block")
]
parsedData2 = parsedData[
    (parsedData["Tag"] != "PacketResponder 0")
    & (parsedData["Tag"] != "PacketResponder 1")
    & (parsedData["Tag"] != "PacketResponder 2")
    & (parsedData["Tag"] != "BLOCK* NameSystem.addStoredBlock:")
    & (parsedData["Tag"] != "BLOCK* NameSystem.delete:")
    & (parsedData["Tag"] != "BLOCK* NameSystem.allocateBlock:")
    & (parsedData["Tag"] != "BLOCK* ask")
    & (parsedData["Tag"] != "Received block")
    & (parsedData["Tag"] != "Receiving block")
    & (parsedData["Tag"] != "Verification succeeded")
    & (parsedData["Tag"] != "Deleting block")
]
parsedData2["Tag"] = parsedData2["Tag"].astype(str)
parsedData2["TypeOfContent"] = parsedData2["TypeOfContent"].astype(str)
parsedData2_fix = parsedData2[parsedData2["Tag"].str.contains("exception")]
parsedData2_fix["Tag"] = "Got exception"
parsedData2_fix["TypeOfContent"] = "Got exception"
parsedData2_fix_2 = parsedData2[parsedData2["TypeOfContent"].str.contains("Served")]
parsedData2_fix_2["Tag"] = "Served"
parsedData2_fix_2["TypeOfContent"] = "Served"
parsedData2_fix_3 = parsedData2[parsedData2["TypeOfContent"].str.contains("Starting")]
parsedData2_fix_3["Tag"] = "Starting thread"
parsedData2_fix_3["TypeOfContent"] = "Starting thread"

frames = [parsedData1, parsedData2_fix, parsedData2_fix_2, parsedData2_fix_3]

parsedData = pd.concat(frames)
parsedData["Date"] = "20" + parsedData["Date"]
parsedData["Date"] = pd.to_datetime(parsedData["Date"], format="%Y%m%d", errors="ignore")     
        
parsedData_2 = parsedData.sort_values(by=['Date', 'Time'])
print(parsedData_2)

#Export to .txt due to file size limit
parsedData_2.to_csv('HDFS_2k-parsed_3.txt', sep='\t', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

                                                     Date  ...                                Tag
1                                                20081109  ...   BLOCK* NameSystem.allocateBlock:
2                                                20081109  ...                    Receiving block
3                                                20081109  ...                    Receiving block
4                                                20081109  ...                  PacketResponder 1
5                                                20081109  ...                  PacketResponder 2
...                                                   ...  ...                                ...
259972                                           20081109  ...  BLOCK* NameSystem.addStoredBlock:
259973                                           20081109  ...  BLOCK* NameSystem.addStoredBlock:
259974                                           20081109  ...  BLOCK* NameSystem.addStoredBlock:
259975              

In [2]:
df = pd.read_csv('HDFS_2k-parsed_3.txt', delimiter = "\t")
df.dropna(inplace=True)
df.tail()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Time,PID,Level,Component,Content,TypeOfContent,datetime_id,Tag
259622,20081109,210916,27.0,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,BLOCK* NameSystem.addStoredBlock:,081109210916,BLOCK* NameSystem.addStoredBlock:
259623,20081109,210916,27.0,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,BLOCK* NameSystem.addStoredBlock:,081109210916,BLOCK* NameSystem.addStoredBlock:
259624,20081109,210916,27.0,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,BLOCK* NameSystem.addStoredBlock:,081109210916,BLOCK* NameSystem.addStoredBlock:
259625,20081109,210916,27.0,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,BLOCK* NameSystem.addStoredBlock:,081109210916,BLOCK* NameSystem.addStoredBlock:
259626,20081109,210916,27.0,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,BLOCK* NameSystem.addStoredBlock:,081109210916,BLOCK* NameSystem.addStoredBlock:


In [3]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df["Level_code"] = ord_enc.fit_transform(df[["Level"]])
print(ord_enc.categories_)
df["Component_code"] = ord_enc.fit_transform(df[["Component"]])
print(ord_enc.categories_)
df["TypeOfContent_code"] = ord_enc.fit_transform(df[["TypeOfContent"]])
print(ord_enc.categories_)
df["Tag_code"] = ord_enc.fit_transform(df[["Tag"]])
print(ord_enc.categories_)

[array(['INFO', 'WARN'], dtype=object)]
[array(['dfs.DataBlockScanner', 'dfs.DataNode', 'dfs.DataNode$DataXceiver',
       'dfs.DataNode$PacketResponder', 'dfs.FSDataset',
       'dfs.FSNamesystem'], dtype=object)]
[array(['BLOCK* NameSystem.addStoredBlock:',
       'BLOCK* NameSystem.allocateBlock:', 'BLOCK* ask', 'Deleting block',
       'PacketResponder 0', 'PacketResponder 1', 'PacketResponder 2',
       'Received block', 'Receiving block', 'Served', 'Starting thread',
       'Verification succeeded'], dtype=object)]
[array(['BLOCK* NameSystem.addStoredBlock:',
       'BLOCK* NameSystem.allocateBlock:', 'BLOCK* ask', 'Deleting block',
       'PacketResponder 0', 'PacketResponder 1', 'PacketResponder 2',
       'Received block', 'Receiving block', 'Served', 'Starting thread',
       'Verification succeeded'], dtype=object)]


In [7]:
#Remove block without block ID
df.drop(df.tail(1).index,inplace=True)

# add block ID to dataframe
listBlocks = [itm[0] for itm in df['Content'].str.findall(r'\sblk\D+\d+') if len(itm)>0]
listBlocksStriped = [i.strip(' ') for i in listBlocks]
df['blkID'] = listBlocksStriped

In [8]:
# Group rows with identical blkIDs
df_comp = df[['Tag_code','blkID']]
df_comp['Tag_code'] = df_comp['Tag_code'].astype(int).astype(str)
df_merged = df_comp[['blkID','Tag_code']].groupby('blkID')['Tag_code'].apply(' '.join).reset_index()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
# Append column to label anomallies and normal data logs
AnomallyLabel = pd.read_csv('AnomallyLabels.csv')
AnomallyLabel = AnomallyLabel['BlockId,Label'].str.split(",",expand=True,)
AnomallyLabel.rename(columns={0:'blockID', 1:"Label"}, inplace = True)
dfLabelled = pd.merge(df_merged,AnomallyLabel, left_on = 'blkID', right_on='blockID')
df_initialAnalysis = pd.merge(df,AnomallyLabel, left_on = 'blkID', right_on='blockID')
df_initialAnalysis.to_csv('HDFS_2k-parsed_3-initialAnalysis.csv', index=False)
dfLabelled = dfLabelled.drop(columns=['blockID'])
dfLabelled.head()

In [11]:
#Convert dataframe into count matrix
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df_filt = df_merged[['blkID', 'Tag_code','Label']]
df_filt["Label"] = ord_enc.fit_transform(df_filt[["Label"]]).astype(int)
df_filt['Tag_code'] = [' '.join(map(str, l)) for l in df_merged['Tag_code']]
df_filt.tail()

In [12]:
df_NN=df_filt.drop(['blkID'], axis=1)
df_NN.rename(columns = {'Tag_code':'review','Label':'sentiment'}, inplace = True)
df_NN.to_csv('HDFS_2k-parsed_3-compl.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,blkID,Label,tag_code0,tag_code1,tag_code2,tag_code3,tag_code4,tag_code5,tag_code6,tag_code7,tag_code8,tag_code9,tag_code10,tag_code11
0,blk_-1000639647761179183,Normal,3,1,0,0,1,1,1,3,3,0,0,0
1,blk_-1000927344760357676,Normal,3,1,0,0,1,1,1,3,3,0,0,0
2,blk_-1000974751560899194,Normal,3,1,0,0,1,1,1,3,3,0,0,0
3,blk_-1001553972418305662,Normal,3,1,0,0,1,1,1,3,3,0,0,0
4,blk_-1001893162880667454,Normal,3,1,0,0,1,1,1,3,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20253,blk_996253271455147639,Normal,3,1,0,0,1,1,1,3,3,0,0,0
20254,blk_997889685682822249,Normal,3,1,0,0,1,1,1,3,3,0,0,0
20255,blk_998616522916827124,Normal,3,1,0,0,1,1,1,3,3,0,0,0
20256,blk_998788538234313747,Normal,3,1,0,0,1,1,1,3,3,0,0,0


In [ ]:
# Create design matrix for 'R' neuro net and PCA_RF file
def createCol(inputDF, outputDF, colName):
    colList = list(set((' '.join(inputDF[colName].astype(str).unique().tolist())).split(' ')))
    colNum = [int(num) for num in colList]
    colNum.sort()
    for col in colNum:
        outputDF['tag_code%s' % col] = 0

    for i in range(len(inputDF)):
        valueSplit = inputDF[colName].iloc[i].split(' ')
        counts = [[str(x),valueSplit.count(str(x))] for x in colNum]
        for val in counts:
            outputDF.loc[i,'tag_code%s' % val[0]] = val[1]

    return outputDF

In [ ]:
df_NNR = df_initialAnalysis[['blkID','Label']]
createCol(df_merged,df_NNR,'Tag_code')
df_NNR.to_csv('R_NN.txt', sep='\t', index=False)